In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.stats
import pandas as pd

In [2]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [8]:
# correlation_filtering()

# se define la funcion
def correlation_filtering(data_frame, treshold):

    df = pd.DataFrame(data_frame)
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # guarda las columnas que se van a tirar
    to_drop =[]

    for n in range(df.shape[1]): # itera sobre columnas del data frame
        
        if any(upper.iloc[:,n]>treshold):     # si algun elemento de la matriz de correlacion de la columna n es mayor al treshold
            print(upper.columns[n], " no pasa el umbral")
            to_drop.append(df.columns[n])     # se agrega para borrarse
            
    print('columns dropped: ', to_drop)            # columnas que se van a tirar   

    # Drop features 
    df.drop(columns= to_drop,inplace=True)

    return df

In [ ]:
# Fast Correlation Based filtering

dfx = pd.DataFrame(X) # base que NO incluye la variable objetivo

# 1. Obtener la correlación de cada variable en X con la variable de salida y guardar las que pasen el umbral
treshold = 0.5
corr_xy = [] # aquí se van a guardar las correlaciones de cada xi con y
high_corr = [] # aquí se guardan las variables que están por arriba del umbral

for n in range(dfx.shape[1]): # itera por columnas del data frame
    column = dfx.iloc[:,n]    # guarda la columna
    a = abs(np.corrcoef(column, y)[0,1]) # calcula la correlacion en valor absoluto
    corr_xy.append(a)                
    if a > treshold:                     
        high_corr.append(dfx.columns[n])
    
print(corr_xy) 
print(high_corr)

# 2. Ordenar las variables de mayor a menor correlación con Y
temp = pd.DataFrame(corr_xy).reset_index()
temp = temp.rename(columns={0: 'corr'})
temp = temp.sort_values(by=['corr'], ascending=False)
print(temp)  

# 3. Medir la correlación de la mejor variable con las demás
corr_matrix = pd_fill_diagonal(dfx.corr().abs())
print(corr_matrix)

# 4. Tirar variables que estén muy correlacionadas con la primera variable.
to_drop = []   # vector de variables que se van a borrar
intocables = [] # vector de variables que NO se deben borrar, porque son las más correlacionadas con y.
for index, row in temp.iterrows():
    # si corr(xi,y) supera el umbral y es la primera variable  and row['corr'] == temp['corr'].max()
    if row['corr']>treshold:     
        
#        print(row['corr'])
#        print(row['index'])
#        print(index)

        # itera por columnas de la matriz de correlacion
        for column in range(corr_matrix.shape[1]):
        
            # se va sobre el renglon de las variables por encima del umbral evaluando cada valor contra el treshold
            if corr_matrix.values[index,column] > treshold:
                print("Row ", index, "Column ", column, "valor ", corr_matrix.values[index,column])

                # para iteraciones después de la primera, esta linea busca evitar que se borre la "mejor" variable.
                # en el ejemplo, el punto es evitar que al iterar sobre la columna 5, se borre la 12.
                if dfx.columns[index] not in intocables:    
                    to_drop.append(dfx.columns[column])    
        
        intocables.append(dfx.columns[index])  # aquí se agregan las que No deben ser tocadas.
        

print('se eliminan las columnas: ', to_drop)
dfx.drop(columns= to_drop,inplace=True)

In [9]:
correlation_filtering(X_train, 0.3)

2  no pasa el umbral
4  no pasa el umbral
5  no pasa el umbral
6  no pasa el umbral
7  no pasa el umbral
8  no pasa el umbral
9  no pasa el umbral
10  no pasa el umbral
11  no pasa el umbral
12  no pasa el umbral
columns dropped:  [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]


,0,1,3
0,0.06129,20.0,1.0
1,8.49213,0.0,0.0
2,1.61282,0.0,0.0
3,0.15098,0.0,0.0
4,0.72580,0.0,0.0
...,...,...,...
374,2.44668,0.0,0.0
375,0.05059,0.0,0.0
376,0.44178,0.0,0.0
377,23.64820,0.0,0.0
